# Borrados lógicos en SQL Server

Retos a solventar en los borrados lógicos:

1. Todas las consultas deben filtrar los registros no válidos.
2. Los índices que se creen deben considerar los borrados.
3. La estructura física en BBDD hay que mantenerla.

## Filtrado de registros

Para filtrar los registros hay varias opciones:

- Si los usuarios acceden directamente a las tablas, debes inculcarles que no se olviden del predicado.
- Si puedes gestionar los accesos, puedes crear vistas sobre las tablas, que incluyan el filtro deseado. Es más legible el código, pero el Database Developer puede olvidar que hay borrados lógicos y quizás se olvide ciertas columnas al crear índices.

## Indexación para borrados lógicos

Para comprender bien esta sección necesitas asentar primero los fundamentos de indexación. Los índices son muy parecidos en todos los SGBD transaccionales, por lo que si vienes de PostgreSQL, Oracle, mySQL, MariaDB, etc. tienes mucho terreno recorrido.

- Arquitectura de índices en SQL Server (por [Microsoft](https://learn.microsoft.com/en-us/sql/relational-databases/sql-server-index-design-guide?view=sql-server-ver16)). Muy completo (y denso). La biblia! 
- Arquitectura de índices en SQL Server; basado en el libro de Kalen Delaney; más corto (enlace a [Medium](https://medium.com/@mohammed0hamdan/sql-server-index-design-and-architecture-part-1-acb9a76937c0))
- Estructura de páginas, y extensiones en SQL Server (por [Microsoft](https://learn.microsoft.com/en-us/sql/relational-databases/pages-and-extents-architecture-guide?view=sql-server-ver16)).

- Indexing best practices (por [Quest](https://blog.quest.com/11-sql-server-index-best-practices-for-improved-performance-tuning/)).
- Consideraciones sobre indices (por [SQLHack](https://www.sqlshack.com/top-five-considerations-for-sql-server-index-design/))
- Indexing tips (por [Idera](https://www.idera.com/resource-center/whitepapers/sql-server-index-performance-tips/))

Como habrás visto de los recursos anteriores, no hay índices buenos por definición. Los índices necesitan dos contextos:   

- Su distribución (cómo de únicas son las columnas que cubren).
- Las consultas que los necesitan. 

En el caso que estamos hablando la columna borrado se necesitará para:

- Cambiar estado de registros (activo a borrado).
    - Estas consultas son muy dirigidas a través de la PK; por tanto muy orientada a operaciones Seek.
- Búsqueda de registros activos.
    - Estas consultas suelen ser variables. Dependen de otras columnas del predicado; por ejemplo, 
        - buscar registros por fecha (que además estén activos).
        - buscar expedientes de un cliente (que estén activos).
        - buscar detalles de un registro concreto (dirigida también por la PK).
    - Ten en cuenta que en estas búsquedas los propios índices ya son eficientes, porque si está activo o no un registro, filtra muy poco.
- Búsqueda de registros borrados para hacer purgados.
    - Esta búsqueda es un poco más exigente y quizás requiera un indice dedicado por el estado, aunque generalmente necesitarás apoyarte en otra columna para hacer el purgado/limpieza semanal/diario. Fíjate que si es un proceso periodico, seguramente tengas una columna fecha de apoyo.
- En qué momento del tiempo el registro pasó a borrado.
    - Quizás necesites saber cuando el registro pasó de activo a borrado. Quizás sea un requerimiento que apareció una vez puesto todo en producción, Quizás, ...
    - Para este caso, un recurso muy útil es añadir una columna fecha (fecha\_borrado por ejemplo), que registre cuando se hizo el borrado. En lugar de cambiar el código de aplicaciones para soportar esta funcionalidad, un TRIGGER de tipo UPDATE te puede ayudar:
        - En el cuerpo del trigger, filtra si la columna borrado se ha actualizado ([ejemplo](https://database.guide/how-to-execute-a-trigger-only-when-a-specific-column-is-updated-sql-server/)).
        - Cuando esa columnas se haya actualizado, y su nuevo estado sea borrado, haz un update de la columna fecha\_borrado con la fecha actual ([getdate()](https://learn.microsoft.com/en-us/sql/t-sql/functions/getdate-transact-sql?view=sql-server-ver16)).

## Mantenimiento de estructura en BBDD

El mantenimento no es complicado. Ten en cuenta que el índice de borrados debería ser bajo. En cualqueir caso, puedes conocer cuantos borrados hay haciendo un GROUP BY por la columna de borrado. Además, quizás te ayude para tener contexto de tiempo, añadir una parte de la fecha (semana, día, mes, etc.) ([Datepart](https://learn.microsoft.com/en-us/sql/t-sql/functions/datepart-transact-sql?view=sql-server-ver16)).

Los scripts de mantenimiento de Ola Hallengren son muy valiosos ([enlace scripts](https://ola.hallengren.com/)). He visto casos con problemas de fragmentación cuando los índices no se han elegido de forma apropiada, pero es más una cuestión de formación y conocimiento del funcionamietno de los indices, que de la funcionalidad de borrado en sí.

## Purgado periódico de datos

La funcionalidad de tu aplicación debe definir con qué frecuencia se borran definitivamente los datos. Concepto parecido a la papelera de reciclaje y su borrado definitivo. Este tema suele ser una necesidad de mantenimiento periódico que debería hacerse de forma periódica. Hacerlo es sencillo. Tendrás que:

- Identificar las tablas del proceso.
- Buscar relaciones (claves ajenas).
- Buscar el criterio de borrado: generalmente columnas externas al proceso; por ejemplo, expedientes que están marcados como borrados y tienen más de xx meses.
- Buscar su cobertura de indexación para que el proceso sea eficiente. 

En función de ello, considera que quizás necesites un índice de cobertura filtrado ([FILTERED INDEXES](https://learn.microsoft.com/en-us/sql/relational-databases/indexes/create-filtered-indexes?view=sql-server-ver16)). 

Incluso quizás sea oportuno crear el índice para la ejecución del proceso, y una vez finalizado el proceso borrar el índice porque no tendrá uso hasta la siguiente ejecución.

Esto recurso es especialmente valioso si trabajas con SQL Server Enterprise ([Operaciones en línea](https://learn.microsoft.com/en-us/sql/relational-databases/indexes/perform-index-operations-online?view=sql-server-ver16)).

  

## Curso Verne Academy

- Curso Avanzado de Transact SQL ([Verne Academy](https://verneacademy.com/courses/curso-t-sql-avanzado/)).

## Referencias de libros

- Indexing for Performnace ([Amazon](https://www.amazon.com/Expert-Performance-Indexing-SQL-Server/dp/1484211197)).
- Cualquiera del resto de libros de [Grant Fritchey](https://www.amazon.com/Grant-Fritchey/e/B002F8BIAQ/ref=dp_byline_cont_book_2)<span style="background-color: rgb(255, 255, 255);">.</span>
- SQL Server Internals de Kalen Delaney ([Amazon](https://www.amazon.com/Microsoft-SQL-Server-2012-Internals-ebook/dp/B00JDMQJYC)).